In [1]:
import os 
import sys
from importlib import reload

# Import Pytorch & Huggingface ==========================
#set cache dir before imports 
from configuration import huggingface_path as cache_dir
#cache_dir = cache_path_here
os.environ['HF_HOME'] = f"{cache_dir}/huggingface"
os.environ['TRANSFORMERS_CACHE']= f"{cache_dir}/huggingface/hub"
os.environ['TORCH_HOME']= f"{cache_dir}/torch"
os.environ['XDG_CACHE_HOME']= cache_dir

#imports 
import torch
from torch.cuda import memory_allocated, empty_cache
from datasets import load_dataset
from torch.utils.data import DataLoader

# Import local functions ==================================
import activation_extractor
reload(activation_extractor)

/u/area/evillegas/.conda/envs/pytorch_env/lib/python3.12/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


<module 'activation_extractor' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/__init__.py'>

In [2]:
#install editable version of this repository
#!python -m pip install -e .

In [3]:
#load data
sequences = ["AAAAAAAAAAA", "HHHHHHHHHHHHHH"]

In [4]:
#load model
model_name = "facebook/esm2_t6_8M_UR50D" 
inferencer = activation_extractor.Inferencer(model_name, device='cpu', half=False)

In [14]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
tokens_ids = inferencer.tokenize(sequences)
outputs = inferencer.inference(tokens_ids)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [15]:
extractor.get_outputs()

{'esm.embeddings': tensor([[[-0.1720, -0.2421, -0.1315,  ...,  0.1121, -0.0918, -0.0569],
          [-0.0587, -0.0105,  0.0076,  ...,  0.0129, -0.0387, -0.1703],
          [-0.0587, -0.0105,  0.0076,  ...,  0.0129, -0.0387, -0.1703],
          ...,
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000]],
 
         [[-0.1720, -0.2421, -0.1315,  ...,  0.1121, -0.0918, -0.0569],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          ...,
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [ 0.0643, -0.2711, -0.1030,  ..., -0.1941, -0.3021, -0.0014]]]),
 'esm.encoder.layer.0.output': tensor([[[ 3.9853e-01,  3.9741e-01, -3.3041e-02,  .